In [ ]:
from math import inf

ROWS, COLS = 7, 8
HUMAN, AI = 1, 2

def create_board():
    return [[0]*COLS for _ in range(ROWS)]

def show_board(board):
    "Print the board with row 0 at the top and labels."
    for r in range(ROWS-1, -1, -1):
        for c in range(COLS):
            cell = board[r][c]
            if cell == 0:    print('.', end=' ')
            elif cell == HUMAN: print('X', end=' ')
            else:           print('O', 
                                  end=' ')
        print()
    print(" " + " ".join(str(i+1) for i in range(COLS)))

def valid_moves(board):
    return [c for c in range(COLS) if board[ROWS-1][c] == 0]

def make_move(board, col, player):
    "Drop a disc into column col (0-index); return False if full."
    for r in range(ROWS):
        if board[r][col] == 0:
            board[r][col] = player
            return True
    return False

def evaluate_and_undo_move(board, col):
    "Evaluate the move aRemove the top disc from column col. if the move initiated is not the best move"
    for r in range(ROWS-1, -1, -1):
        if board[r][col] != 0:
            board[r][col] = 0
            return True
    return False

def check_win(board, player):
    "Check for four in a row for player."
    # Horizontal
    for r in range(ROWS):
        for c in range(COLS-3):
            if all(board[r][c+i] == player for i in range(4)):
                return True
    # Vertical
    for c in range(COLS):
        for r in range(ROWS-3):
            if all(board[r+i][c] == player for i in range(4)):
                return True
    # Diagonal /
    for r in range(ROWS-3):
        for c in range(COLS-3):
            if all(board[r+i][c+i] == player for i in range(4)):
                return True
    # Diagonal \
    for r in range(3, ROWS):
        for c in range(COLS-3):
            if all(board[r-i][c+i] == player for i in range(4)):
                return True
    return False

def is_draw(board):
    return all(board[ROWS-1][c] != 0 for c in range(COLS))

def evaluate_window(window):
    "Score a window of 4 cells for the computer."
    score = 0
    if window.count(AI) == 4:
        score += 100
    elif window.count(AI) == 3 and window.count(0) == 1:
        score += 5
    elif window.count(AI) == 2 and window.count(0) == 2:
        score += 2
    if window.count(HUMAN) == 3 and window.count(0) == 1:
        score -= 4
    return score

def score_position(board):
    "Compute overall board score for AI."
    score = 0
    # Horizontal windows
    for r in range(ROWS):
        for c in range(COLS-3):
            score += evaluate_window([board[r][c+i] for i in range(4)])
    # Vertical windows
    for c in range(COLS):
        for r in range(ROWS-3):
            score += evaluate_window([board[r+i][c] for i in range(4)])
    # Diagonal windows
    for r in range(ROWS-3):
        for c in range(COLS-3):
            score += evaluate_window([board[r+i][c+i] for i in range(4)])
    for r in range(3, ROWS):
        for c in range(COLS-3):
            score += evaluate_window([board[r-i][c+i] for i in range(4)])
    return score

# Global counters for node counts
minimax_nodes = 0
alphabeta_nodes = 0

def minimax(board, depth, maximizing):
    "Minimax algorithm (no pruning) returns (best_col, score)."
    global minimax_nodes
    minimax_nodes += 1

    if check_win(board, AI):    return (None,  1000000)
    if check_win(board, HUMAN): return (None, -1000000)
    if is_draw(board):         return (None, 0)
    if depth == 0:
        return (None, score_position(board))

    valid = valid_moves(board)
    if maximizing:
        value = -inf; best_col = valid[0]
        for col in valid:
            make_move(board, col, AI)
            new_score = minimax(board, depth-1, False)[1]
            evaluate_and_undo_move(board, col)
            if new_score > value:
                value = new_score; best_col = col
        return best_col, value
    else:
        value = inf; best_col = valid[0]
        for col in valid:
            make_move(board, col, HUMAN)
            new_score = minimax(board, depth-1, True)[1]
            evaluate_and_undo_move(board, col)
            if new_score < value:
                value = new_score; best_col = col
        return best_col, value

def minimax_ab(board, depth, alpha, beta, maximizing):
    "Minimax with alpha-beta pruning."
    global alphabeta_nodes
    alphabeta_nodes += 1

    if check_win(board, AI):    return (None,  1000000)
    if check_win(board, HUMAN): return (None, -1000000)
    if is_draw(board):         return (None, 0)
    if depth == 0:
        return (None, score_position(board))

    valid = valid_moves(board)
    if maximizing:
        value = -inf; best_col = valid[0]
        for col in valid:
            make_move(board, col, AI)
            new_score = minimax_ab(board, depth-1, alpha, beta, False)[1]
            evaluate_and_undo_move(board, col)
            if new_score > value:
                value = new_score; best_col = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break  # prune
        return best_col, value
    else:
        value = inf; best_col = valid[0]
        for col in valid:
            make_move(board, col, HUMAN)
            new_score = minimax_ab(board, depth-1, alpha, beta, True)[1]
            evaluate_and_undo_move(board, col)
            if new_score < value:
                value = new_score; best_col = col
            beta = min(beta, value)
            if alpha >= beta:
                break  # prune
        return best_col, value

def play_connect4():
    board = create_board()
    turn = HUMAN  # Human goes first
    while True:
        show_board(board)
        if turn == HUMAN:
            col = int(input("Enter column (1-8) for your move: ")) - 1
            if col < 0 or col >= COLS or not make_move(board, col, HUMAN):
                print("Invalid move. Try again.")
                continue
            if check_win(board, HUMAN):
                show_board(board)
                print("You win!")
                break
        else:
            # Compare performance: we show nodes for one decision
            global minimax_nodes, alphabeta_nodes
            minimax_nodes = alphabeta_nodes = 0
            # Compute move via Minimax (depth=3)
            col_mm, _ = minimax(board, 3, True)
            # Compute move via Alpha-Beta
            col_ab, _ = minimax_ab(board, 3, -inf, inf, True)
            print(f"\n[Debug] Minimax nodes: {minimax_nodes}, AlphaBeta nodes: {alphabeta_nodes}")
            # They should agree on the move
            best_col = col_ab
            make_move(board, best_col, AI)
            print(f"Computer plays column {best_col+1}.")
            if check_win(board, AI):
                show_board(board)
                print("Computer Wins!")
                break
            elif check_win(board,HUMAN):
                show_board(board)
                print("Human Wins!")
        # Check for draw
        if is_draw(board):
            show_board(board)
            print("The game is a draw.")
            break
        turn = HUMAN if turn == AI else AI

In [ ]:
play_connect4()